In [2]:
import pandas as pd

In [3]:
data_file = 'capture20110815-2.pcap.netflow.labeled'

infected = '147.32.84.165'
n = 10
k_s = [100, 1000, 5000]
columns=['Date','Duration','Protocol','SrcIP', 'srcPort','DstIP', 'DstPort', 'Flags', 'TOS', 'Packets', 'Bytes', 'Label']
lst=[]
with open(data_file) as fp:
    for cnt, line in enumerate(fp):
        if cnt!=0:
            data=line.split("\t")
            data = list(filter(None, data)) 
            date = data[0]
            duration = data[1]
            protocol = data[2]
            src = data[3].split(':')
            if len(src) < 2: # Set port to 0 if no port provided
                src.append(0)
            dst = data[5].split(':')
            if len(dst) < 2:
                dst.append(0)
            flags = data[6]
            tos = data[7]
            packets = data[8]
            byteno = data[9]
            flows = data[10]
            label = data[11].rstrip()
            lst.append([date, duration, protocol, src[0], src[1], dst[0], dst[1], flags, tos, packets, byteno, label])
dataset=pd.DataFrame(lst, columns=columns)
dataset.head()

,Date,Duration,Protocol,SrcIP,srcPort,DstIP,DstPort,Flags,TOS,Packets,Bytes,Label
0,2011-08-15 16:43:20.931,0.235,TCP,147.32.84.59,64131,164.8.32.159,54992,PA_,0,3,211,Background
1,2011-08-15 16:43:20.933,0.444,TCP,147.32.87.252,49218,147.32.80.13,3128,FPA_,0,300,19566,LEGITIMATE
2,2011-08-15 16:43:20.933,4.993,TCP,41.143.58.184,1328,147.32.86.179,58675,PA_,0,269,29373,Background
3,2011-08-15 16:43:20.933,0.187,TCP,147.32.86.155,51273,204.154.94.81,80,FA_,0,2,132,Background
4,2011-08-15 16:43:20.933,0.442,TCP,147.32.80.13,3128,147.32.87.252,49218,FPA_,0,579,728343,LEGITIMATE
